In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torchvision
from torch.utils.data import DataLoader,random_split,Subset
from torchvision import datasets,transforms
import matplotlib.pyplot as plt

### Constants And Transforms

In [68]:
RANDOM_SEED=123 #ensure reproducibility by controling randomnes in training
BATHC_SIZE=256 # no of images into the model at each training step
NUM_EPOCHS=50 # No of complete passes through the training dataset
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [69]:
# def set_all_seeds(seed):
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed) # for multi- GPU
#     np.RANDOM_SEED(seed)
#     RANDOM_SEED(seed)
#     # cuDNN! may use non-deterministic algorithms that run faster but may give slightly different results each time you run the code
#     torch.backend.cudnn.deterministic=True
#     torch.backends.cudnn.benchmark = False #banchmark =want reproducible and stable results, or when your input sizes change frequently.



In [70]:

def set_all_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using multi-GPU
    np.random.seed(seed)
    random.seed(seed)
 # cuDNN! may use non-deterministic algorithms that run faster but may give slightly different results each time you run the code
    torch.backends.cudnn.deterministic = True
 #banchmark =want reproducible and stable results, or when your input sizes change frequently.
    torch.backends.cudnn.benchmark = False


In [71]:
RANDOM_SEED = 123
set_all_seeds(RANDOM_SEED)


### Data Augmentation & Normalization

In [72]:
train_transforms=torchvision.transforms.Compose([
    torchvision.transforms.Resize((70,70)),# resize the images 70x70
    torchvision.transforms.RandomCrop((64,64)), # randomly crop 64x64 data augmentation
    torchvision.transforms.ToTensor(), # converts image to pyTorch tensor[0,1]
    torchvision.transforms.Normalize(0.5,0.5,0.5),(0.5,0.5,0.5)]) # shift pixel value from [0,1]to [-1,1] > mean=0.5,std=0.5

In [73]:
test_transforms=torchvision.transforms.Compose([
    torchvision.transforms.Resize((70,70)),
    torchvision.transforms.RandomCrop((64,64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(0.5,0.5,0.5),(0.5,0.5,0.5)])

### Loading CIFAR-10 Data

In [83]:
def get_dataloaders_cifar10(batch_size, validation_fraction, train_transforms, test_transforms, num_workers=2):
   # Download CIFAR-10 train and test datasets
    train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=train_transforms)
    test_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=test_transforms)

    # Create a validation split from the training dataset
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(validation_fraction * num_train))

    np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]

    train_data = Subset(train_dataset, train_idx)
    valid_data = Subset(train_dataset, valid_idx)

    # Create data loaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, valid_loader, test_loader

In [84]:
train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=256,
    validation_fraction=0.1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    num_workers=2
)

In [93]:
# for images, labels in train_loader:
#     print('Image batch dimensions:', images.shape)
#     print('Image label dimensions:', labels.shape)
#     print('Class labels of 10 examples:', labels[:10])
#     break


### Model

### Build Convolutional Blocks(1-5)

In [101]:

class VGG16(torch.nn.Module):
    def __init__(self,num_classes):
      super().__init__()


      self.block_1=torch.nn.Sequential(
    torch.nn.conv2d(3,64,kernel_size=3,padding=1), # input (RGB Image)3x64x64
    torch.nn.RELU(),
    torch.nn.Conv2d(64,64,kernel_size=3,padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2,stride=2))
      # Two conv layers with 64 filters, followed by ReLU, then max pooling.

      self.block_2=torch.nn.Sequential(
          torch.nn.Conv2d(64,128,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Conve2d(128,128,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2,stride=2))


      self.block_3=torch.nn.Sequential(
          torch.nn.Conv2d(128,256,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Conv2d(256,256,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Conv2d(256,256,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Maxpool2d(kernel_size=2,stride=2))


      self.block_4=torch.nn.Sequential(
          torch.nn.Conv2d(256,512,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Conv2d(512,512,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Conv2d(512,512,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Maxpool2d(kernel_size=2,stride=2))


      self.block_5=torch.nn.Sequential(
          torch.nn.Conv2d(512,512,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Conv2d(512,512,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Conv2d(512,512,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.Maxpool2d(kernel_size=2,stride=2))


      self.avgpool=torch.nn.AdaptiveAvgpool2d((3,3))



In [113]:
# self.classifier = torch.nn.Sequential(
#     torch.nn.Linear(512*3*3, 4096),
#     torch.nn.ReLU(True),
#     torch.nn.Dropout(p=0.5),
#     torch.nn.Linear(4096, 4096),
#     torch.nn.ReLU(True),
#     torch.nn.Dropout(p=0.5),
#     torch.nn.Linear(4096, num_classes),
# )

### Classifier

In [106]:
class VGG16(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # ... other blocks ...

        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(512 * 3 * 3, 4096),
            torch.nn.ReLU(True),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(True),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(4096, num_classes)  # use it directly here
        )


In [108]:
class VGG16(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # ... define your blocks and classifier here ...

        # Initialize weights for Conv and Linear layers
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
                torch.nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    m.bias.detach().zero_()


In [109]:
def forward(self, x):
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.block_3(x)
    x = self.block_4(x)
    x = self.block_5(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1) # flatten to (batch_size, 512*3*3)
    logits = self.classifier(x)
    return logits


In [111]:
model = VGG16(num_classes=10)
print(sum(p.numel() for p in model.parameters()))  # should print a large number > 0


0
